In [1]:
import numpy as np
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem, rdDistGeom
import py3Dmol

In [2]:
def MolTo3DView(mol, size=(300, 300), style="stick", surface=False, opacity=0.5):
    """Draw molecule in 3D
    
    Args:
    ----
        mol: rdMol, molecule to show
        size: tuple(int, int), canvas size
        style: str, type of drawing molecule
               style can be 'line', 'stick', 'sphere', 'carton'
        surface, bool, display SAS
        opacity, float, opacity of surface, range 0.0-1.0
    Return:
    ----
        viewer: py3Dmol.view, a class for constructing embedded 3Dmol.js views in ipython notebooks.
    """
    assert style in ('line', 'stick', 'sphere', 'cartoon')
    mblock = Chem.MolToMolBlock(mol)
    viewer = py3Dmol.view(width=size[0], height=size[1])
    viewer.addModel(mblock, 'mol')
    viewer.setStyle({style:{'colorscheme':'cyanCarbon'}})
    if surface:
        viewer.addSurface(py3Dmol.SAS, {'opacity': opacity})
    viewer.zoomTo()
    return viewer


def smi2conf(smiles, path, offset):
    '''Convert SMILES to rdkit.Mol with 3D coordinates'''
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        mol = Chem.AddHs(mol)
        ps = AllChem.ETKDG()
        distmat = np.loadtxt(path)
        num_atoms = distmat.shape[0]
        for i in range(num_atoms):
            for j in range(num_atoms):
                if j > i:
                    distmat[i][j] += offset
                    distmat[j][i] -= offset
        ps.SetBoundsMat(distmat)
        AllChem.EmbedMolecule(mol, params=ps)
        
        return mol
    else:
        return None

In [3]:
smiles = "CC"
path = "distmat-0.txt"
out = "conf-distmat-0.txt"
offset = 0.0005
img_height = 600
img_width = 600
img_style = "stick"

In [5]:
conf = smi2conf(smiles, path, offset)
viewer = MolTo3DView(conf, size=(img_height, img_width), style=img_style)
viewer.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol